In [1]:
# First import library
import pyrealsense2 as rs
# Import Numpy for easy array manipulation
import numpy as np
# Import OpenCV for easy image rendering
import cv2
import open3d as o3d
from datetime import datetime

def get_intrinsic_matrix(frame):
    intrinsics = frame.profile.as_video_stream_profile().intrinsics
    out = o3d.camera.PinholeCameraIntrinsic(640, 480, intrinsics.fx,
                                            intrinsics.fy, intrinsics.ppx,
                                            intrinsics.ppy)
    return out


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create pipeline
pipeline = rs.pipeline()

# Create a config object
config = rs.config()

# Tell config that we will use a recorded device from file to be used by the pipeline through playback.
rs.config.enable_device_from_file(config, "20230404_112125.bag")
#rs.config.enable_device_from_file(config, "20230403_112453.bag")

In [ ]:
config.enable_stream(rs.stream.depth, rs.format.z16, 30)
pipeline.start(config)

#cv2.namedWindow("Depth Stream", cv2.WINDOW_AUTOSIZE)
vis = o3d.visualization.Visualizer()
vis.create_window()

view_control = vis.get_view_control()
view_control.set_lookat([2.6172, 2.0475, 1.532])
view_control.set_front([0.4257, -0.2125, -0.8795])
view_control.set_zoom(0.3412)
view_control.set_up([-0.0694, -0.9768, 0.2024])

colourizer = rs.colorizer()

count = 0

while True:
    frames = pipeline.wait_for_frames()
    print(f"Got frame {frames}")
    depth_frame = frames.get_depth_frame()
    colour_frame = frames.get_color_frame()
    
    intrinsic = o3d.camera.PinholeCameraIntrinsic(
                    get_intrinsic_matrix(depth_frame))
    print(f"Colour frame {colour_frame}")
    if colour_frame:
        print("OK frame")
    else:
        print("Bad frame")

    #if not colour_frame:
    #    colour_frame = o3d.geometry.Image()
    if colour_frame:
        print("Yeah - valid colour frame!")
        intrinsic = o3d.camera.PinholeCameraIntrinsic(
                        get_intrinsic_matrix(colour_frame))
        #depth_image = o3d.geometry.Image(
        #                  np.array(aligned_depth_frame.get_data()))
        color_temp = np.array(color_frame.get_data())
        color_image = o3d.geometry.Image(color_temp)

        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
                color_image,
                depth_image,
                depth_scale=1.0 / depth_scale,
                depth_trunc=clipping_distance_in_meters,
                convert_rgb_to_intensity=False)
        temp = o3d.geometry.PointCloud.create_from_rgbd_image(
                rgbd_image, intrinsic)
        #o3d.visualization.draw_geometries([temp],zoom=0.5)
        o3d.visualization.update_geometry(temp)

    depth_colour_frame = colourizer.colorize(depth_frame)

    align_to = rs.stream.color
    align = rs.align(align_to)

    # Align the depth frame to color frame
    aligned_frames = align.process(frames)

    # Get aligned frames
    aligned_depth_frame = aligned_frames.get_depth_frame()
    depth_image = o3d.geometry.Image(np.array(aligned_depth_frame.get_data()))

    pcd = o3d.geometry.PointCloud.create_from_depth_image(depth_image, intrinsic,
                                                         np.identity(4), depth_scale=1000.0, depth_trunc=3000.0)

    mean,cov = pcd.compute_mean_and_covariance()
    
    print(f"Point cloud {count}: {mean} {cov} {pcd}")
    # This is supposed to work - but the pcd is the wrong type?
    #o3d.visualization.draw_geometries([pcd],zoom=0.5)
    if count > -1:
        # This blocks until the window is closed
        o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=mean,
                                  up=[-0.0694, -0.9768, 0.2024])
    else:
        view_control = vis.get_view_control()
        view_control.set_lookat(mean)
        view_control.set_front([0.4257, -0.2125, -0.8795])
        view_control.set_zoom(0.3412)
        view_control.set_up([-0.0694, -0.9768, 0.2024])

        vis.update_geometry(pcd)
        t0 = datetime.now()
        t = datetime.now()-t0
        while t.total_seconds() < 3.0:
            vis.poll_events()
            vis.update_renderer()
            t = datetime.now()-t0
    count = count + 1
    #o3d.visualization.draw_geometries([temp],zoom=0.5)

    #depth_image = o3d.geometry.Image(
    #            np.array(aligned_depth_frame.get_data()))
    #depth_colour_frame = colourizer.colorize(depth_frame)

    #cv2.show("Depth ",depth_colour_frame)
    print(f"Got frame {depth_frame}")
    #key = cv2.waitKey(1)
    key = 0
    if key == 27:
            cv2.destroyAllWindows()
            break

Got frame <pyrealsense2.frameset Z16 #775 @1680621685700.294189>
Colour frame <pyrealsense2.frame NULL>
Bad frame
Point cloud 0: [ 0.53204195 -0.42962216  2.09788313] [[ 1.57161762  0.02980434  0.75493601]
 [ 0.02980434  0.6169142  -1.16434055]
 [ 0.75493601 -1.16434055  4.63260725]] PointCloud with 338421 points.
